# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*.

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-s4nr3e_v
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-s4nr3e_v
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fee45af12ee0af490cd8e091514173b571dcd9ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-4bws0n9z/elegantrl_8fb54c276c9e469ab52db8e76f061dc6
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-4bws0n9z/elegantrl_8fb54c276c9e469ab52db8e76f061dc6
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 95a1a6bbc92e62c8d2ac470b2315d961da343889
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/1

In [2]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [3]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
aapl_df_yf.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.468269,72.528589,71.223267,71.476607,135480400
2020-01-03,71.763725,72.523754,71.539337,71.696167,146322800
2020-01-06,72.335548,72.374154,70.634532,70.885464,118387200
2020-01-07,71.995354,72.600960,71.775789,72.345204,108872000
2020-01-08,73.153496,73.455095,71.698581,71.698581,132079200


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [5]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)
Shape of DataFrame:  (20, 8)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
aapl_df_finrl.head()

Price,date,close,high,low,open,volume,tic,day
0,2020-01-02,72.468269,72.528589,71.223267,71.476607,135480400,aapl,3
1,2020-01-03,71.763725,72.523754,71.539337,71.696167,146322800,aapl,4
2,2020-01-06,72.335548,72.374154,70.634532,70.885464,118387200,aapl,0
3,2020-01-07,71.995354,72.600960,71.775789,72.345204,108872000,aapl,1
4,2020-01-08,73.153496,73.455095,71.698581,71.698581,132079200,aapl,2


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Data for the chosen tickers

In [7]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [8]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [9]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for re

Shape of DataFrame:  (91072, 8)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2009-01-02,2.721687,2.730385,2.554038,2.575631,746015200,AAPL,4
1,2009-01-02,40.176647,40.237944,39.332112,39.904216,6547900,AMGN,4
2,2009-01-02,14.821132,14.966813,14.108061,14.238407,10955700,AXP,4
3,2009-01-02,33.941093,34.173619,32.088396,32.103398,7010200,BA,4
4,2009-01-02,30.147322,30.192308,28.733463,28.861996,7117200,CAT,4


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [11]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (3228, 8)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Successfully added vix


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Successfully added turbulence index


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [12]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [13]:
processed_full.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.721687,2.730385,2.554038,2.575631,746015200.0,4.0,0.0,2.941564,2.616675,100.0,66.666667,100.0,2.721687,2.721687,39.189999,0.0
1,2009-01-02,AMGN,40.176647,40.237944,39.332112,39.904216,6547900.0,4.0,0.0,2.941564,2.616675,100.0,66.666667,100.0,40.176647,40.176647,39.189999,0.0
2,2009-01-02,AXP,14.821132,14.966813,14.108061,14.238407,10955700.0,4.0,0.0,2.941564,2.616675,100.0,66.666667,100.0,14.821132,14.821132,39.189999,0.0
3,2009-01-02,BA,33.941093,34.173619,32.088396,32.103398,7010200.0,4.0,0.0,2.941564,2.616675,100.0,66.666667,100.0,33.941093,33.941093,39.189999,0.0
4,2009-01-02,CAT,30.147322,30.192308,28.733463,28.861996,7117200.0,4.0,0.0,2.941564,2.616675,100.0,66.666667,100.0,30.147322,30.147322,39.189999,0.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Part 4: Save the Data

### Split the data for training and trading

In [14]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

81004
9380


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [15]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')

# Task
I will now proceed with the following steps:
1. Mount Google Drive.
2. Save `train_data.csv` to Google Drive.
3. Save `trade_data.csv` to Google Drive.
4. Confirm that the files have been successfully saved to the specified Google Drive location.

Proceeding with mounting Google Drive.

## Mount Google Drive

### Subtask:
Mount Google Drive to access its file system. This step requires user interaction to grant permissions.


**Reasoning**:
To mount Google Drive, I need to import the `drive` module from `google.colab` and then call its `mount()` method with the specified mount point.



In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Save `train_data.csv` to Google Drive

### Subtask:
Save the `train_data.csv` DataFrame to a CSV file in a designated folder within Google Drive.

#### Instructions
1. Specify the path to the desired saving location within your mounted Google Drive.
2. Use the `to_csv()` method on the `train` DataFrame to save it to the specified path.

**Reasoning**:
To save the `train` DataFrame to Google Drive, I need to define the save path and then use the `to_csv()` method. It's also good practice to ensure the directory exists before saving.



In [17]:
import os

drive_path = '/content/gdrive/MyDrive/FinRL_Data/'
os.makedirs(drive_path, exist_ok=True)

train.to_csv(f'{drive_path}train_data.csv', index=False)
print(f"'train_data.csv' saved to {drive_path}")

'train_data.csv' saved to /content/gdrive/MyDrive/FinRL_Data/


## Save `trade_data.csv` to Google Drive

### Subtask:
Save the `trade_data.csv` DataFrame to a CSV file in the same designated folder within Google Drive.

#### Instructions
1. Use the `to_csv()` method on the `trade` DataFrame to save it to the specified path.

**Reasoning**:
To save the `trade` DataFrame to Google Drive, I will use the `to_csv()` method with the previously defined `drive_path`.



In [18]:
trade.to_csv(f'{drive_path}trade_data.csv', index=False)
print(f"'trade_data.csv' saved to {drive_path}")

'trade_data.csv' saved to /content/gdrive/MyDrive/FinRL_Data/


## Confirm Files Saved to Google Drive

### Subtask:
Verify that `train_data.csv` and `trade_data.csv` are present in the specified Google Drive folder.

#### Instructions
1. List the contents of the `drive_path` directory to confirm the presence of the saved CSV files.

**Reasoning**:
To confirm the files are saved, I will list the contents of the Google Drive path where the files were supposedly saved and check for their presence.



In [19]:
import os

# Ensure drive_path is accessible from previous cells or re-define if necessary
# drive_path = '/content/gdrive/MyDrive/FinRL_Data/'

# List contents of the directory
print(f"Contents of {drive_path}:")
print(os.listdir(drive_path))

# Verify files are present
expected_files = ['train_data.csv', 'trade_data.csv']
found_files = [f for f in expected_files if f in os.listdir(drive_path)]

if len(found_files) == len(expected_files):
    print("All expected files confirmed in Google Drive.")
else:
    print(f"Missing files: {set(expected_files) - set(found_files)}")

Contents of /content/gdrive/MyDrive/FinRL_Data/:
['train_data.csv', 'trade_data.csv']
All expected files confirmed in Google Drive.


## Summary:

### Data Analysis Key Findings
*   Google Drive was successfully mounted at `/content/gdrive`.
*   The `train_data.csv` file was successfully saved to `/content/gdrive/MyDrive/FinRL_Data/`.
*   The `trade_data.csv` file was successfully saved to `/content/gdrive/MyDrive/FinRL_Data/`.
*   Both `train_data.csv` and `trade_data.csv` were confirmed to be present in the specified Google Drive folder.

### Insights or Next Steps
*   The data persistence achieved by saving the CSV files to Google Drive ensures that the processed data is readily available for future sessions or collaborative work without reprocessing.
*   The saved data can now be easily accessed for subsequent steps in the FinRL project, such as model training or backtesting, directly from Google Drive.
